In [1]:
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [5]:
def get_train_data(num_samples=1000):
    num_classes = 10
    (x_train, y_train), _= keras.datasets.mnist.load_data()
    
    idx = np.random.choice(np.arange(len(x_train)), num_samples, replace=True)
    x_train = x_train[idx]
    y_train = y_train[idx]
    
    x_train = x_train.astype("float32") / 255
    x_train = np.expand_dims(x_train, -1)
    y_train = keras.utils.to_categorical(y_train, num_classes)
    return (x_train, y_train)

def get_test_data(num_samples=1000):
    num_classes = 10
    _, (x_test, y_test) = keras.datasets.mnist.load_data()
    
    idx = np.random.choice(np.arange(len(x_test)), num_samples, replace=True)
    x_test = x_test[idx]
    y_test = y_test[idx]
    
    x_test = x_test.astype("float32") / 255
    x_test = np.expand_dims(x_test, -1)
    y_test = keras.utils.to_categorical(y_test, num_classes)
    return (x_test, y_test)

In [6]:
(x_train, y_train) = get_train_data()
(x_test, y_test) = get_test_data()

In [9]:
batch_size = 128
epochs = 20
input_shape = (28, 28, 1)
num_classes = 10

model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
    )

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(x_train, y_train, epochs = epochs)

Epoch 1/20
32/32 [==============================] - 0s 10ms/step - loss: 2.0883 - accuracy: 0.2500
Epoch 2/20
32/32 [==============================] - 0s 10ms/step - loss: 1.0913 - accuracy: 0.6700
Epoch 3/20
32/32 [==============================] - 0s 11ms/step - loss: 0.5995 - accuracy: 0.7920
Epoch 4/20
32/32 [==============================] - 0s 11ms/step - loss: 0.3975 - accuracy: 0.8700
Epoch 5/20
32/32 [==============================] - 0s 11ms/step - loss: 0.3235 - accuracy: 0.9010
Epoch 6/20
32/32 [==============================] - 0s 11ms/step - loss: 0.2911 - accuracy: 0.9110
Epoch 7/20
32/32 [==============================] - 0s 10ms/step - loss: 0.2527 - accuracy: 0.9280
Epoch 8/20
32/32 [==============================] - 0s 11ms/step - loss: 0.2153 - accuracy: 0.9330
Epoch 9/20
32/32 [==============================] - 0s 11ms/step - loss: 0.1673 - accuracy: 0.9480
Epoch 10/20
32/32 [==============================] - 0s 10ms/step - loss: 0.1424 - accuracy: 0.9610
Epoch 11/

In [19]:
probs = model.predict(x_test)

In [51]:
predictions = np.argmax(probs, axis=1)

unique, counts = np.unique(predictions, return_counts=True)
unique = unique.tolist()
counts = counts.tolist()

pred_counts = dict(zip(unique, counts))
pred_counts

{0: 95, 1: 125, 2: 115, 3: 115, 4: 84, 5: 95, 6: 96, 7: 88, 8: 93, 9: 94}

In [59]:
from datetime import datetime
import json, codecs

timestamp = datetime.now()
date_time = timestamp.strftime("%m/%d/%Y, %H:%M:%S")

data = {"timestamp": date_time, "pred_counts": pred_counts}

with open('results.txt', 'a', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

In [82]:
from datetime import datetime
import json, codecs

for i in range(1):
    (x_test, y_test) = get_test_data()
    
    probs = model.predict(x_test)
    
    predictions = np.argmax(probs, axis=1)

    unique, counts = np.unique(predictions, return_counts=True)
    unique = unique.tolist()
    counts = counts.tolist()

    pred_counts = dict(zip(unique, counts))
    
    timestamp = datetime.now()
    date_time = timestamp.strftime("%m/%d/%Y, %H:%M:%S")

    data = {"timestamp": date_time, "pred_counts": pred_counts}

    with open('results.txt', 'a', encoding='utf-8') as f:
        json_data = json.load(f)
        json_data.append(data)
        json.dump(data, f, ensure_ascii=False, indent=4)

UnsupportedOperation: not readable

In [97]:
from datetime import datetime
import csv

for i in range(3):
    (x_test, y_test) = get_test_data()
    
    probs = model.predict(x_test)
    
    predictions = np.argmax(probs, axis=1)

    unique, counts = np.unique(predictions, return_counts=True)
    unique = unique.tolist()
    counts = counts.tolist()

    pred_counts = dict(zip(unique, counts))
    
    timestamp = datetime.now()
    date_time = timestamp.strftime("%m/%d/%Y, %H:%M:%S")

    data = {"timestamp": date_time, "pred_counts": pred_counts}

    with open('results.csv', 'a', newline='') as f:
        writer = csv.writer(f, delimiter="|")
        writer.writerow([date_time, pred_counts])

In [98]:
def store_inference_results(probs):
    from datetime import datetime
    import csv

    predictions = np.argmax(probs, axis=1)

    unique, counts = np.unique(predictions, return_counts=True)
    unique = unique.tolist()
    counts = counts.tolist()

    pred_counts = dict(zip(unique, counts))

    timestamp = datetime.now()
    date_time = timestamp.strftime("%m/%d/%Y, %H:%M:%S")

    with open('results.csv', 'a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([date_time, pred_counts])

In [2]:
def federated_decorator(func):
    """
    Returns a wrapped function that can be deployed to funcx endpoints when given endpoint_ids

    Parameters
    ----------
    func: function
        the function to be deployed to funcx endpoints

    Returns
    -------
    wrapper: function
        the wrapped function func
    
    """
    def wrapper(*args, **kwargs):
        """
        Returns a wrapped function that is deployed to specified funcx endpoints.

        Parameters
        ----------
        *args: positional arguments

        **kwargs: keyword arguments
            when using a wrapped function, you need to pass in parameters as 
            keywords

        Returns
        -------
        tasks: list
            contains funcx task objects from which the original result can be retrieved with
            task[i].result()

        """
        fx = FuncXExecutor(FuncXClient())
        tasks = []

        # for each endpoint, submit the function with **kwargs to it
        for e in kwargs["endpoint_ids"]:
            tasks.append(fx.submit(func, 
                                   **kwargs,
                                    endpoint_id=e))
        return tasks
    
    return wrapper

In [13]:
@federated_decorator
def retrieve_inference_results(path_dir, filename):
    import os
    import csv
    
    # construct the path
    results_file = os.sep.join([path_dir, filename])
    
    rows = []
    with open(results_file, 'r') as f:
        reader = csv.reader(f, delimiter='|')
        for row in reader:
            rows.append(row)
        
    return rows

In [9]:
fx = FuncXExecutor(FuncXClient())

cur_path = '/home/pi/globus'
name = 'results.csv'

tasks = []

# for each endpoint, submit the function with **kwargs to it
for e in endpoint_ids:
    tasks.append(fx.submit(retrieve_inference_results, 
                           path_dir=cur_path,
                           filename=name,
                            endpoint_id=e))

[NON-CRITICAL] Off_process_checker instantiation failed. Continuing...
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\funcx\serialize\facade.py", line 32, in __init__
    port = self._start_off_process_checker()
  File "C:\ProgramData\Anaconda3\lib\site-packages\funcx\serialize\facade.py", line 60, in _start_off_process_checker
    ["off_process_checker.py"], stdout=std_out, stderr=subprocess.STDOUT
  File "C:\ProgramData\Anaconda3\lib\subprocess.py", line 800, in __init__
    restore_signals, start_new_session)
  File "C:\ProgramData\Anaconda3\lib\subprocess.py", line 1207, in _execute_child
    startupinfo)
OSError: [WinError 193] %1 is not a valid Win32 application


In [12]:
tasks[0].result()

[['yeey', 'data'], ['anotheryyer', 'moredata']]

In [14]:
import os
from funcx.sdk.client import FuncXClient
from funcx.sdk.executor import FuncXExecutor

#cur_path = os.getcwd()
cur_path = '/home/pi/globus'
name = 'results.csv'

endpoint_ids = ["6d2cc03e-565d-494b-9bdf-0ba0acdc606f", "11983ca1-2d45-40d1-b5a2-8736b3544dea"]

results = retrieve_inference_results(path_dir=cur_path, filename=name, endpoint_ids=endpoint_ids)

[NON-CRITICAL] Off_process_checker instantiation failed. Continuing...
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\funcx\serialize\facade.py", line 32, in __init__
    port = self._start_off_process_checker()
  File "C:\ProgramData\Anaconda3\lib\site-packages\funcx\serialize\facade.py", line 60, in _start_off_process_checker
    ["off_process_checker.py"], stdout=std_out, stderr=subprocess.STDOUT
  File "C:\ProgramData\Anaconda3\lib\subprocess.py", line 800, in __init__
    restore_signals, start_new_session)
  File "C:\ProgramData\Anaconda3\lib\subprocess.py", line 1207, in _execute_child
    startupinfo)
OSError: [WinError 193] %1 is not a valid Win32 application
Caught unexpected exception while setting results
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\funcx\sdk\asynchronous\ws_polling_task.py", line 200, in set_result
    data["exception"]
  File "C:\ProgramData\Anaconda3\lib\site-packages\funcx\s